Thanks to these great works:
  - [motono0223: js24-train-gbdt-model-with-lags-singlemodel](https://www.kaggle.com/code/motono0223/js24-train-gbdt-model-with-lags-singlemodel)
  - [yuanzhezhou: jane-street-baseline-lgb-xgb-and-catboost](https://www.kaggle.com/code/yuanzhezhou/jane-street-baseline-lgb-xgb-and-catboost)

This notebook mainly includes the following contents:

How to perform feature engineering with lagged N date_ids (utilizing shift and rolling operations)
How to dynamically store data of the most recent N date_ids for updating lag features

The model employed herein is a single-fold XGBoost model incorporated with lag-1 (previous day) features, which achieved a score of 0.0052 on the Leaderboard (LB).

When time_id=0 of each new date_id arrives, a lag label corresponding to the previous date_id of the current date_id is provided. For instance, when date_id=100, the label for date_id=99 is assigned; however, the date_id in this lag data remains 100 (to align with the current main dataset). This lag data needs to be combined with the dynamically stored most recent N days of data to construct additional shift and rolling features.

In [1]:
!pip install rtdl_num_embeddings -q --no-index --find-links=/kaggle/input/jane-street-import/rtdl_num_embeddings

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
import pandas as pd
import math
import numpy as np
from tqdm import tqdm
import polars as pl
from collections import OrderedDict
import sys
from tabm_reference import Model, make_parameter_groups

import warnings
warnings.filterwarnings("ignore")

import kaggle_evaluation.jane_street_inference_server

import os

import joblib

from pytorch_lightning import LightningModule

In [3]:
feature_list = [f"feature_{idx:02d}" for idx in range(79) if idx != 61]

target_col = "responder_6" 

feature_test = feature_list \
                + [f"responder_{idx}_lag_1" for idx in range(9)] 

feature_cat = ["feature_09", "feature_10", "feature_11"]
feature_cont = [item for item in feature_test if item not in feature_cat]

batch_size = 8192

std_feature = [i for i in feature_list if i not in feature_cat] + [f"responder_{idx}_lag_1" for idx in range(9)]

data_stats = joblib.load("/kaggle/input/my-own-js/data_stats.pkl")
means = data_stats['mean']
stds = data_stats['std']

def standardize(df, feature_cols, means, stds):
    return df.with_columns([
        ((pl.col(col) - means[col]) / stds[col]).alias(col) for col in feature_cols
    ])

In [4]:
category_mappings = {'feature_09': {2: 0, 4: 1, 9: 2, 11: 3, 12: 4, 14: 5, 15: 6, 25: 7, 26: 8, 30: 9, 34: 10, 42: 11, 44: 12, 46: 13, 49: 14, 50: 15, 57: 16, 64: 17, 68: 18, 70: 19, 81: 20, 82: 21},
 'feature_10': {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 10: 7, 12: 8},
 'feature_11': {9: 0, 11: 1, 13: 2, 16: 3, 24: 4, 25: 5, 34: 6, 40: 7, 48: 8, 50: 9, 59: 10, 62: 11, 63: 12, 66: 13,
  76: 14, 150: 15, 158: 16, 159: 17, 171: 18, 195: 19, 214: 20, 230: 21, 261: 22, 297: 23, 336: 24, 376: 25, 388: 26, 410: 27, 522: 28, 534: 29, 539: 30},
 'symbol_id': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19,
  20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38},
 'time_id' : {i : i for i in range(968)}}

def encode_column(df, column, mapping):
    max_value = max(mapping.values())  

    def encode_category(category):
        return mapping.get(category, max_value + 1)  
    
    return df.with_columns(
        pl.col(column).map_elements(encode_category).alias(column)
    )

In [5]:
class R2Loss(nn.Module):
    def __init__(self):
        super(R2Loss, self).__init__()

    def forward(self, y_pred, y_true):
        mse_loss = torch.sum((y_pred - y_true) ** 2)
        var_y = torch.sum(y_true ** 2)
        loss = mse_loss / (var_y + 1e-38)
        return loss

class NN(LightningModule):
    def __init__(self, n_cont_features, cat_cardinalities, n_classes, lr, weight_decay):
        super().__init__()
        self.save_hyperparameters()
        self.k = 16
        self.model = Model(
                n_num_features=n_cont_features,
                cat_cardinalities=cat_cardinalities,
                n_classes=n_classes,
                backbone={
                    'type': 'MLP',
                    'n_blocks': 3 ,
                    'd_block': 512,
                    'dropout': 0.25,
                },
                bins=None,
                num_embeddings= None,
                arch_type='tabm',
                k=self.k,
            )
        self.lr = lr
        self.weight_decay = weight_decay
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.loss_fn = R2Loss()
        # self.loss_fn = weighted_mse_loss

    def forward(self, x_cont, x_cat):
        return self.model(x_cont, x_cat).squeeze(-1)

    def training_step(self, batch):
        x_cont,x_cat, y, w , w_y= batch
        x_cont = x_cont + torch.randn_like(x_cont) * 0.02
        y_hat = self(x_cont, x_cat)
        # loss = self.loss_fn(y_hat.flatten(0, 1), y.repeat_interleave(self.k), w_y.repeat_interleave(self.k))
        loss = self.loss_fn(y_hat.flatten(0, 1), y.repeat_interleave(self.k))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, batch_size=x_cont.size(0))
        self.training_step_outputs.append((y_hat.mean(1), y, w))
        return loss

    def validation_step(self, batch):
        x_cont,x_cat, y, w, w_y = batch
        x_cont = x_cont + torch.randn_like(x_cont) * 0.02
        y_hat = self(x_cont, x_cat)
        # loss = self.loss_fn(y_hat.flatten(0, 1), y.repeat_interleave(self.k), w_y.repeat_interleave(self.k))
        loss = self.loss_fn(y_hat.flatten(0, 1), y.repeat_interleave(self.k))
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True, batch_size=x_cont.size(0))
        self.validation_step_outputs.append((y_hat.mean(1), y, w))
        return loss

    def on_validation_epoch_end(self):
        """Calculate validation WRMSE at the end of the epoch."""
        y = torch.cat([x[1] for x in self.validation_step_outputs]).cpu().numpy()
        if self.trainer.sanity_checking:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
        else:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
            weights = torch.cat([x[2] for x in self.validation_step_outputs]).cpu().numpy()
            # r2_val
            val_r_square = r2_val(y, prob, weights)
            self.log("val_r_square", val_r_square, prog_bar=True, on_step=False, on_epoch=True)
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(make_parameter_groups(self.model), lr=self.lr, weight_decay=self.weight_decay)
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5,
        #                                                        verbose=True)
        return {
            'optimizer': optimizer,
            # 'lr_scheduler': {
            #     'scheduler': scheduler,
            #     'monitor': 'val_r_square',
            # }
        }

    def on_train_epoch_end(self):
        if self.trainer.sanity_checking:
            return

        y = torch.cat([x[1] for x in self.training_step_outputs]).cpu().numpy()
        prob = torch.cat([x[0] for x in self.training_step_outputs]).detach().cpu().numpy()
        weights = torch.cat([x[2] for x in self.training_step_outputs]).cpu().numpy()
        # r2_training
        train_r_square = r2_val(y, prob, weights)
        self.log("train_r_square", train_r_square, prog_bar=True, on_step=False, on_epoch=True)
        self.training_step_outputs.clear()

        epoch = self.trainer.current_epoch
        metrics = {k: v.item() if isinstance(v, torch.Tensor) else v for k, v in self.trainer.logged_metrics.items()}
        formatted_metrics = {k: f"{v:.5f}" for k, v in metrics.items()}
        print(f"Epoch {epoch}: {formatted_metrics}")
        
class custom_args():
    def __init__(self):
        self.usegpu = True
        self.gpuid = 0
        self.seed = 42
        self.model = 'nn'
        self.use_wandb = False
        self.project = 'js-tabm-with-lags'
        self.dname = "./input_df/"
        self.loader_workers = 10   
        self.bs = 8192
        self.lr = 1e-3
        self.weight_decay = 8e-4
        self.n_cont_features = 84
        self.n_cat_features = 5
        self.n_classes = None
        self.cat_cardinalities = [23, 10, 32, 40, 969]
        self.patience = 7
        self.max_epochs = 10
        self.N_fold = 5


my_args = custom_args()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

modelnn = NN.load_from_checkpoint('/kaggle/input/my-own-js/tabm_epochepoch03.ckpt').to(device)


In [6]:
lags_ : pl.DataFrame | None = None

lags_history = None

def predictnn(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    global lags_, lags_history
    if lags is not None:
        lags_ = lags
    
    for col in feature_cat + ['symbol_id', 'time_id']:
        test = encode_column(test, col, category_mappings[col])

    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )
    
    symbol_ids = test.select('symbol_id').to_numpy()[:, 0]

    time_id = test.select("time_id").to_numpy()[0]
    timie_id_array = test.select("time_id").to_numpy()[:, 0]
    
    
    if time_id == 0:
        lags = lags.with_columns(pl.col('time_id').cast(pl.Int64))
        lags = lags.with_columns(pl.col('symbol_id').cast(pl.Int64))
    
        lags_history = lags
        lags = lags.filter(pl.col("time_id") == 0)
        
        
        test = test.join(lags, on=["time_id", "symbol_id"],  how="left")
    else:
        lags = lags_history.filter(pl.col("time_id") == time_id)
        test = test.join(lags, on=["time_id", "symbol_id"],  how="left")

    
    test = test.with_columns([
        pl.col(col).fill_null(0) for col in feature_list + [f"responder_{idx}_lag_1" for idx in range(9)] 
    ])

    test = standardize(test, std_feature, means, stds)


    X_test = test[feature_test].to_numpy()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

    symbol_tensor = torch.tensor(symbol_ids, dtype=torch.float32).to(device)
    time_tensor = torch.tensor(timie_id_array, dtype=torch.float32).to(device)
    X_cat = X_test_tensor[:, [9, 10, 11]]
    X_cont = X_test_tensor[:, [i for i in range(X_test_tensor.shape[1]) if i not in [9, 10, 11]]]
    # X_cont = X_cont + torch.randn_like(X_cont) * 0.02

    X_cat = (torch.concat([X_cat, symbol_tensor.unsqueeze(-1), time_tensor.unsqueeze(-1)], axis=1)).to(torch.int64)
    

    modelnn.eval()
    with torch.no_grad():
        
        outputs = modelnn(X_cont, X_cat)
        # Assuming the model outputs a tensor of shape (batch_size, 1)
        preds = outputs.squeeze(-1).cpu().numpy()
        preds = preds.mean(1)
    
    
    # predictions = \
    # test.select('row_id').\
    # with_columns(
    #     pl.Series(
    #         name   = 'responder_6', 
    #         values = np.clip(preds, a_min = -5, a_max = 5),
    #         dtype  = pl.Float64,
    #     )
    # )


    # # The predict function must return a DataFrame
    # assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    # # with columns 'row_id', 'responer_6'
    # assert list(predictions.columns) == ['row_id', 'responder_6']
    # # and as many rows as the test data.
    # assert len(predictions) == len(test)

    return preds

In [7]:
import pandas as pd
import polars as pl
import numpy as np
import os, gc
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import pickle

from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import kaggle_evaluation.jane_street_inference_server

pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(400)
pl.Config.set_fmt_table_cell_list_len(5)

polars.config.Config

# Configurations

In [8]:
class CONFIG:
    debug = False
    seed = 42
    target_col = "responder_6"
    lag_cols_rename = { f"responder_{idx}_lag_1" : f"responder_{idx}" for idx in range(9)}
    lag_target_cols_name = [f"responder_{idx}" for idx in range(9)]
    lag_cols_original = ["date_id", "time_id", "symbol_id"] + [f"responder_{idx}" for idx in range(9)]
    model_path = "/kaggle/input/janestreet-public-model/xgb_001.pkl"
    lag_ndays = 4

In [9]:
def create_agg_list(day, columns):
    agg_mean_list = [pl.col(c).mean().name.suffix(f"_mean_{day}d") for c in columns]
    agg_std_list = [pl.col(c).std().name.suffix(f"_std_{day}d") for c in columns]
    agg_max_list = [pl.col(c).max().name.suffix(f"_max_{day}d") for c in columns]
    agg_last_list = [pl.col(c).last().name.suffix(f"_last_{day}d") for c in columns]
    agg_list = agg_mean_list + agg_std_list + agg_max_list + agg_last_list
    return agg_list

# Load model

In [10]:
with open( CONFIG.model_path, "rb") as fp:
    result = pickle.load(fp)
    
model = result["model"]
features = result["features"]
print(len(features))

116


Prior to performing inference on the test set, it is necessary to preliminarily construct historical data. The key considerations are as follows:
  - If lag features of N days are utilized, the last N days of data in the training set must first be stored.
  - The date_id in the test set starts from 0, so it is required to pre-adjust the date_ids of the training set and the test set to ensure consistency.
  - Currently, it is unclear whether the test set is immediately subsequent to the training set. Herein, we first adjust the date_ids of the training set to be consistent with those of the test set (i.e., -N, -N+1, ..., -2, -1).

In [11]:
history = pl.scan_parquet(
    "/kaggle/input/jane-street-realtime-marketdata-forecasting/train.parquet"
).select(['date_id','time_id','symbol_id'] + [f"responder_{idx}" for idx in range(9)]).filter(
    (pl.col("date_id")>=(1698 - CONFIG.lag_ndays))&(pl.col("date_id")<1698)
)
# 这里将历史date_id变为从-N到-1, 假设test的date_id=0紧随train的date_id=1698,
# 在第一个batch给出的lags应该是date_id=1698的responser(但date_id给的0),
# 这样history中最后一个date_id=1697变为-1, 正好可以和推理时给的lags衔接上
history = history.with_columns(
    date_id = (pl.col("date_id") - pl.lit(1698)).cast(pl.Int16)
)
history = history.collect()

# 这里是为了统一特征的dtypes(polars在concat时如果dtype对不上会报错)
history_column_types = {
    'date_id': pl.Int16,
    'time_id': pl.Int16,
    'symbol_id': pl.Int16
}
feature_column_types = {}
for f in [f"feature_{idx:02d}" for idx in range(79)]:
    feature_column_types[f] = pl.Float32

responder_column_types = {}
for f in [f"responder_{idx}" for idx in range(9)]:
    responder_column_types[f] = pl.Float32

history = history.cast(history_column_types)
history = history.cast(responder_column_types)
history.tail()

date_id,time_id,symbol_id,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
i16,i16,i16,f32,f32,f32,f32,f32,f32,f32,f32,f32
-1,967,34,0.501321,0.905332,-0.819582,-0.564046,-0.223018,-0.283954,-0.045938,0.009797,-0.102538
-1,967,35,-1.113053,0.69719,-1.619031,-1.222743,-0.706082,-0.291133,0.167733,0.099704,0.32461
-1,967,36,-1.019353,-0.460962,-2.026678,-0.848606,-0.305448,-1.256913,-0.109359,-0.027474,-0.253956
-1,967,37,0.23585,0.556479,0.618944,-0.243765,-0.108361,-0.260777,-0.486923,-0.275566,-1.020708
-1,967,38,0.542563,0.513193,0.814393,0.032767,0.025435,0.311465,-0.044797,0.011133,-0.0793


In [12]:
feature_cols2 = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]

xgb_model2 = None
xgb_model2 = lgb.Booster(model_file="/kaggle/input/11111111/lgbm_model_2.json")
xgb_feature_cols = ["symbol_id", "time_id"] + feature_cols2
display(xgb_model2)

In [13]:
# Custom R2 score calculation function
def r2_val(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true) ** 2, weights=sample_weight) / (np.average((y_true) ** 2, weights=sample_weight) + 1e-38) # R2スコア計算
    return r2


class NN3(LightningModule):
    # Definition of the NN model
    def __init__(self, input_dim, hidden_dims, dropouts, lr, weight_decay):
        # ~Arguments~
        # input_dim: Dimension of the input layer
        # hidden_dims: Specify the dimensions of hidden layers as a list
        # dropouts: Argument to specify dropout rates for each hidden layer as a list
        # lr: Learning rate
        # weight_decay: Weight decay for regularization
        
        super().__init__() # Code to call the constructor of LightningModule
        self.save_hyperparameters() # Save the arguments passed to the constructor as hyperparameters
        layers = [] # Empty list to store each layer of the NN
        in_dim = input_dim
        
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.BatchNorm1d(in_dim)) # Add batch normalization layer to the layers list
            if i > 0: # Execute only after the first hidden layer
                layers.append(nn.SiLU()) # Add SiLU activation function to the layers list
            if i < len(dropouts): # Conditional branch to apply dropout layer
                layers.append(nn.Dropout(dropouts[i])) # Add dropout layer to the layers list
            layers.append(nn.Linear(in_dim, hidden_dim)) # Add linear layer to the layers list
            # layers.append(nn.ReLU())
            in_dim = hidden_dim # Set the input dimension for the next layer
        layers.append(nn.Linear(in_dim, 1))  # Add linear layer from input dimension to output dimension 1 as the output layer to the layers list
        layers.append(nn.Tanh()) # Add Tanh as the activation function for the output layer to the layers list
        self.model = nn.Sequential(*layers) # Create an nn.Sequential model that executes layers stored in the list in order and store it in self.model
        self.lr = lr # Store the learning rate passed to the constructor in the self.lr attribute
        self.weight_decay = weight_decay # Store the weight decay passed to the constructor in the self.weight_decay attribute
        self.validation_step_outputs = [] # Initialize empty list to store outputs of the validation step

    # Forward propagation of the NN model
    def forward(self, x):
        return 5 * self.model(x).squeeze(-1)  # Output is a 1-dimensional tensor

    # Training step: Calculate loss function and record logs
    def training_step(self, batch):
        x, y, w = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction='none') * w  # Consider sample weights
        loss = loss.mean() # Average loss
        self.log('train_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
        return loss

    # Validation step: Calculate loss function and record logs
    def validation_step(self, batch):
        x, y, w = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction='none') * w
        loss = loss.mean()
        self.log('val_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
        self.validation_step_outputs.append((y_hat, y, w))
        return loss

    # Execute at the end of the validation epoch: Calculate custom R2 score and record logs
    def on_validation_epoch_end(self):
        """Calculate validation WRMSE at the end of the epoch."""
        y = torch.cat([x[1] for x in self.validation_step_outputs]).cpu().numpy()
        if self.trainer.sanity_checking:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
        else:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
            weights = torch.cat([x[2] for x in self.validation_step_outputs]).cpu().numpy()
            # r2_val
            val_r_square = r2_val(y, prob, weights)
            self.log("val_r_square", val_r_square, prog_bar=True, on_step=False, on_epoch=True)
        self.validation_step_outputs.clear()

    # Set up optimization algorithm and learning rate scheduler
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5,
                                                               verbose=True)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
            }
        }

    # Output logs of training result metrics at the end of the training epoch
    def on_train_epoch_end(self):
        if self.trainer.sanity_checking:
            return
        epoch = self.trainer.current_epoch
        metrics = {k: v.item() if isinstance(v, torch.Tensor) else v for k, v in self.trainer.logged_metrics.items()}
        formatted_metrics = {k: f"{v:.5f}" for k, v in metrics.items()}
        print(f"Epoch {epoch}: {formatted_metrics}")

In [14]:
N_folds = 5 # Set the number of models
# Load Best Model
modelsnn = [] # Initialize list to store models
for fold in range(N_folds): # Perform iterative processing for each fold
    checkpoint_path = f"/kaggle/input/js-xs-nn-trained-model/nn_{fold}.model" # Create checkpoint file path for the model
    modelnn3 = NN3.load_from_checkpoint(checkpoint_path) # Load model from checkpoint file
    modelsnn.append(modelnn3.to("cuda:0")) # Transfer model to GPU and add to the list

In [15]:
lags_ : pl.DataFrame | None = None

In [16]:
CONfeature_cols  = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]

In [19]:

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    global history
    global lags_infer
    global lags_
    if lags is not None:
        lags_ = lags

    pdss=predictnn(test,lags)
    
    symbol_ids = test.select("symbol_id").to_numpy()[:, 0]
    current_date = test.select("date_id").to_numpy()[:, 0][0]

    lags2 = lags_.clone().group_by(["date_id", "symbol_id"], maintain_order=True).last()
    X_test = test.join(lags2, on=["date_id", "symbol_id"],  how="left")
    preds2 = np.zeros((test.shape[0],))
    preds2 += xgb_model2.predict(X_test[xgb_feature_cols].to_numpy())*1.1


    test_input = X_test[CONfeature_cols].to_pandas()
    test_input = test_input.fillna(method = 'ffill').fillna(0)
    test_input = torch.FloatTensor(test_input.values).to("cuda:0")
    preds_nno = np.zeros((X_test.shape[0],))
    with torch.no_grad():
        for i, nn_model3 in enumerate(modelsnn):
            nn_model3.eval()
            preds_nno += nn_model3(test_input).cpu().numpy()/len(modelsnn)
    
    if lags is not None:
        lags = lags.rename(CONFIG.lag_cols_rename)
        lags = lags.cast(history_column_types)
        lags = lags.cast(responder_column_types)

        history = pl.concat([history, lags])
        
        history = history.filter(pl.col("date_id") > (current_date - CONFIG.lag_ndays))
        agg_list = create_agg_list(1, CONFIG.lag_target_cols_name)
        shift_n_data = history.filter(pl.col("date_id") == current_date)
        lags_infer = shift_n_data.group_by(["date_id", "symbol_id"], maintain_order=True).agg(agg_list)
  
    test = test.cast(history_column_types)
    test = test.cast(feature_column_types)
    X_test = test.join(lags_infer, on=["date_id", "symbol_id"], how="left")
    preds = np.zeros((X_test.shape[0],))
    preds += model.predict(X_test[features].to_numpy())*1.05

    pds = preds*0.7 + preds2*0.3
    #pds = np.clip(pds, a_min=-5, a_max=5)

    #print(pdss)
    pdss2=pds*0.55+pdss*0.35+preds_nno*0.2*1.1
    pdss2 = np.clip(pdss2, a_min=-5, a_max=5)
    
    predictions = (
        test.select('row_id').with_columns(
            pl.Series(name='responder_6', values=pdss2, dtype=pl.Float64)
        )
    )

    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)

    return predictions

In [20]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/test.parquet',
            '/kaggle/input/jane-street-realtime-marketdata-forecasting/lags.parquet',
        )
    )